In [1]:
import pandas as pd
import numpy as np

In [2]:
import env

In [3]:
from pydataset import data

# Exercises Part 1

### 3
Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [4]:
def get_db_url(username, password, hostname, database_name):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'

### 4
Use your function to obtain a connection to the employees database.

In [5]:
url = get_db_url(env.username, env.password, env.hostname, 'employees')
employees = pd.read_sql('SELECT * FROM employees;', url)
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


### 5
Once you have successfully run a query:

a. Intentionally make a typo in the database url. What kind of error message do you see?

b. Intentionally make an error in your SQL query. What does the error message look like?

In [6]:
url = get_db_url(env.username, env.password, env.hostname, 'fruits_db')

# This one gives a NoSuchModuleError
# fruits = pd.read_sql('SELECT * FROM fruits;', url[1 : ])

In [7]:
# This one gives an OperationalError
# fruits = pd.read_sql('SELECT * FROM fruits;', url[ : -2])

In [8]:
# This one gives a ProgrammingError along with a syntax error from mysql
# fruits = pd.read_sql('SELECT * fruits;', url)

### 6
Read the employees and titles tables into two separate DataFrames.

In [9]:
employees_url = get_db_url(env.username, env.password, env.hostname, 'employees')
employees = pd.read_sql('SELECT * FROM employees;', employees_url)
titles = pd.read_sql('SELECT * FROM titles;', employees_url)

### 7
How many rows and columns do you have in each DataFrame? Is that what you expected?

In [10]:
employees.shape

(300024, 6)

In [11]:
titles.shape

(443308, 4)

### 8
Display the summary statistics for each DataFrame.

In [12]:
employees.describe(include = 'all')

,emp_no,birth_date,first_name,last_name,gender,hire_date
count,300024.000000,300024,300024,300024,300024,300024
unique,NaN,4750,1275,1637,2,5434
top,NaN,1952-03-08,Shahab,Baba,M,1985-06-20
freq,NaN,95,295,226,179973,132
mean,253321.763392,NaN,NaN,NaN,NaN,NaN
std,161828.235540,NaN,NaN,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN,NaN,NaN
25%,85006.750000,NaN,NaN,NaN,NaN,NaN
50%,249987.500000,NaN,NaN,NaN,NaN,NaN
75%,424993.250000,NaN,NaN,NaN,NaN,NaN


In [13]:
titles.describe(include = 'all')

,emp_no,title,from_date,to_date
count,443308.000000,443308,443308,443308
unique,NaN,7,6393,5888
top,NaN,Engineer,1998-10-25,9999-01-01
freq,NaN,115003,132,240124
mean,253075.034430,NaN,NaN,NaN
std,161853.292613,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN
25%,84855.750000,NaN,NaN,NaN
50%,249847.500000,NaN,NaN,NaN
75%,424891.250000,NaN,NaN,NaN


### 9
How many unique titles are in the titles DataFrame?

In [14]:
titles.title.unique().size

7

### 10
What is the oldest date in the to_date column?

In [15]:
titles.to_date.min()

datetime.date(1985, 3, 1)

### 11
What is the most recent date in the to_date column?

In [16]:
titles[titles.to_date.astype(str) != '9999-01-01'].to_date.max()

datetime.date(2002, 8, 1)

# Exercises Part 2

### 1
Copy the users and roles DataFrames from the examples above.

In [17]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [18]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### 2
What is the result of using a right join on the DataFrames?

In [19]:
users.merge(roles, how = 'right', left_on = 'role_id', right_on = 'id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [37]:
roles.merge(users, how = 'right', left_on = 'id', right_on = 'role_id')

,id_x,name_x,id_y,name_y,role_id
0,1.0,admin,1,bob,1.0
1,2.0,author,2,joe,2.0
2,3.0,reviewer,3,sally,3.0
3,3.0,reviewer,4,adam,3.0
4,NaN,NaN,5,jane,NaN
5,NaN,NaN,6,mike,NaN


### 3
What is the result of using an outer join on the DataFrames?

In [38]:
users.merge(roles, how = 'outer', left_on = 'role_id', right_on = 'id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


### 4
What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [90]:
users.drop(columns = 'role_id').merge(roles)

,id,name


### 5
Load the mpg dataset from PyDataset.

In [23]:
mpg = data('mpg')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


### 6
Output and read the documentation for the mpg dataset.

In [24]:
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




### 7
How many rows and columns are in the dataset?

In [25]:
mpg.shape

(234, 11)

### 8
Check out your column names and perform any cleanup you may want on them.

In [26]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [27]:
mpg = mpg.rename(columns = {'displ' : 'displacement', 'cyl' : 'cylinders', 'drv' : 'drive', 'cty' : 'city', 'hwy' : 'highway', 'class' : 'vehicle_class'})
mpg.columns

Index(['manufacturer', 'model', 'displacement', 'year', 'cylinders', 'trans',
       'drive', 'city', 'highway', 'fl', 'vehicle_class'],
      dtype='object')

### 9
Display the summary statistics for the dataset.

In [28]:
mpg.describe(include = 'all')

,manufacturer,model,displacement,year,cylinders,trans,drive,city,highway,fl,vehicle_class
count,234,234,234.000000,234.000000,234.000000,234,234,234.000000,234.000000,234,234
unique,15,38,NaN,NaN,NaN,10,3,NaN,NaN,5,7
top,dodge,caravan 2wd,NaN,NaN,NaN,auto(l4),f,NaN,NaN,r,suv
freq,37,11,NaN,NaN,NaN,83,106,NaN,NaN,168,62
mean,NaN,NaN,3.471795,2003.500000,5.888889,NaN,NaN,16.858974,23.440171,NaN,NaN
std,NaN,NaN,1.291959,4.509646,1.611534,NaN,NaN,4.255946,5.954643,NaN,NaN
min,NaN,NaN,1.600000,1999.000000,4.000000,NaN,NaN,9.000000,12.000000,NaN,NaN
25%,NaN,NaN,2.400000,1999.000000,4.000000,NaN,NaN,14.000000,18.000000,NaN,NaN
50%,NaN,NaN,3.300000,2003.500000,6.000000,NaN,NaN,17.000000,24.000000,NaN,NaN
75%,NaN,NaN,4.600000,2008.000000,8.000000,NaN,NaN,19.000000,27.000000,NaN,NaN


### 10
How many different manufacturers are there?

In [29]:
mpg.manufacturer.unique().size

15

### 11
How many different models are there?

In [91]:
mpg.model.nunique()

38

### 12
Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [31]:
mpg['mileage_difference'] = (mpg.highway - mpg.city).abs()
mpg.head()

,manufacturer,model,displacement,year,cylinders,trans,drive,city,highway,fl,vehicle_class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10


### 13
Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [32]:
mpg['average_mileage'] = mpg.loc[ : , 'city' : 'highway'].mean(axis = 1)
mpg.head()

,manufacturer,model,displacement,year,cylinders,trans,drive,city,highway,fl,vehicle_class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0


### 14
Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [33]:
mpg.trans.dtype

dtype('O')

In [34]:
is_automatic_bool = mpg.trans.str.contains('auto')
mpg['is_automatic'] = is_automatic_bool
mpg.head()

,manufacturer,model,displacement,year,cylinders,trans,drive,city,highway,fl,vehicle_class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True


### 15
Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [35]:
mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending = False).head(1)

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

In [92]:
mpg.groupby('manufacturer').average_mileage.mean().idxmax()

'honda'

### 16
Do automatic or manual cars have better miles per gallon?

In [95]:
avg_mpg_by_transmission = mpg.groupby('is_automatic').average_mileage.mean()
print(f'Manual: {avg_mpg_by_transmission[False]}\nAutomatic: {avg_mpg_by_transmission[True]}')

Manual: 22.227272727272727
Automatic: 19.130573248407643


# Exercises Part 3

### 1
Use your get_db_url function to help you explore the data from the chipotle database.

In [43]:
chipotle = pd.read_sql('SELECT * FROM orders;', env.get_db_url('chipotle'))
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


### 2
What is the total price for each order?

In [49]:
item_price_as_float = chipotle.item_price.str.lstrip('$').astype(float)
chipotle.assign(item_price_float = item_price_as_float).groupby('order_id').item_price_float.sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price_float, Length: 1834, dtype: float64

### 3
What are the most popular 3 items?

In [51]:
chipotle.groupby('item_name').item_name.count().sort_values(ascending = False).head(3)

item_name
Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

### 4
Which item has produced the most revenue?

In [53]:
chipotle.assign(item_price_float = item_price_as_float).groupby('item_name').item_price_float.sum().sort_values(ascending = False).head(1)

item_name
Chicken Bowl    7342.73
Name: item_price_float, dtype: float64

### 5
Join the employees and titles DataFrames together.

In [57]:
employees_titles = employees.merge(titles, how = 'inner', on = 'emp_no')
employees_titles.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


### 6
For each title, find the hire date of the employee that was hired most recently with that title.

In [58]:
employees_titles.groupby('title').hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

### 7
Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [59]:
current_titles = titles[titles.to_date.astype(str) == '9999-01-01']
current_titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
5,10005,Senior Staff,1996-09-12,9999-01-01


In [85]:
# This query will work and is a good example of a solution in the case that the wildcard character is needed, however the query below is a much better solution
# for this particular situation.

# sql = '''
# SELECT
#     emp_no,
#     dept_name
# FROM dept_emp
# JOIN departments USING (dept_no)
# WHERE to_date LIKE '9999%%';
# '''

sql = '''
SELECT
    emp_no,
    dept_name
FROM dept_emp
JOIN departments USING (dept_no)
WHERE to_date > NOW();
'''

current_emp_departments = pd.read_sql(sql, env.get_db_url('employees'))
current_emp_departments.head()

,emp_no,dept_name
0,10038,Customer Service
1,10049,Customer Service
2,10060,Customer Service
3,10088,Customer Service
4,10112,Customer Service


In [78]:
current_titles_by_department = current_titles[['emp_no', 'title']].merge(current_emp_departments, how = 'inner', on = 'emp_no')
current_titles_by_department.head()

,emp_no,title,dept_name
0,10001,Senior Engineer,Development
1,10002,Staff,Sales
2,10003,Senior Engineer,Production
3,10004,Senior Engineer,Production
4,10005,Senior Staff,Human Resources


In [82]:
pd.crosstab(current_titles_by_department.title, current_titles_by_department.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1652,0,0,0,1402,389,77,0
Engineer,627,14040,0,0,0,12081,3405,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,38816,0,0,0,33625,9458,2250,0
Senior Staff,11268,1085,9545,9824,11290,1123,0,9092,28797
Staff,3574,315,2891,3073,3551,349,0,2870,8903
Technique Leader,241,5477,0,0,0,4723,1293,321,0


In [96]:
pd.crosstab(current_titles_by_department.title, current_titles_by_department.dept_name).style.highlight_max()

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1652,0,0,0,1402,389,77,0
Engineer,627,14040,0,0,0,12081,3405,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,38816,0,0,0,33625,9458,2250,0
Senior Staff,11268,1085,9545,9824,11290,1123,0,9092,28797
Staff,3574,315,2891,3073,3551,349,0,2870,8903
Technique Leader,241,5477,0,0,0,4723,1293,321,0
